In [4]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import fftconvolve
import librosa
from IPython.display import Audio

In [2]:
%config Completer.use_jedi = False

In [50]:
def remove_noise(audio_signal,
                 noise_signal,
                 std_away=1,
                 n_grad_freq=2,
                 n_grad_time=4,
                 decrease_prop=1,
                 n_fft=2048,
                 win_length=2048,
                 hop_length=512):
    
    
    
    noise_stft = librosa.stft(noise_signal,n_fft=n_fft,hop_length=hop_length,win_length=win_length)
    noise_stft_db = librosa.core.amplitude_to_db(np.abs(noise_stft), ref=1.0, amin=1e-20, top_db=80.0)
    mean_noise = np.mean(noise_stft_db,axis=1)
    std_noise = np.mean(noise_stft_db,axis=1)
    noise_thresh = mean_noise + std_away * std_noise
    
    audio_stft = librosa.stft(audio_signal,n_fft=n_fft,hop_length=hop_length,win_length=win_length)
    audio_stft_db = librosa.core.amplitude_to_db(np.abs(audio_stft), ref=1.0, amin=1e-20, top_db=80.0)
    
    mask_gain_db = np.min(audio_stft_db)
    noise_thresh_matrix = np.repeat(noise_thresh.reshape(1,-1),audio_stft_db.shape[1],axis=0).T
    noise_mask = audio_stft_db < noise_thresh_matrix
    
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    
    smoothing_filter = fftconvolve(noise_mask,smoothing_filter,mode='same') * decrease_prop
    
    cleaned_audio_real = audio_stft_db * (1-smoothing_filter) + np.ones(mask_gain_db.shape) * mask_gain_db * smoothing_filter
    cleaned_audio_img = np.imag(audio_stft) * (1-smoothing_filter)
    
    cleaned_audio_stft = librosa.core.db_to_amplitude(cleaned_audio_real)*np.sign(audio_stft) + 1j * cleaned_audio_img
    
    cleaned_audio = librosa.istft(cleaned_audio_stft,hop_length=hop_length,win_length=win_length)

    return cleaned_audio
    
    

In [56]:
def get_noise(datalen,min_freq,max_freq,rate):

    freqs = np.abs(np.fft.fftfreq(n=datalen,d=1/rate))
    samples = np.zeros(datalen)
    samples[np.logical_and(freqs>=min_freq,freqs<=max_freq)]=1

    samples = np.array(samples,dtype='complex')
    Np = (len(samples)-1)//2
    phases = np.random.rand(Np) * 2 * np.pi
    e_iphases = np.cos(phases) + 1j * np.sin(phases)
    samples[1:Np+1] *= e_iphases
    samples[Np+1:-1] = np.conj(samples[1:Np+1])
    noise = (np.fft.ifft(samples).real)

    return noise

In [65]:
rate,audio = wavfile.read("/Users/shahules/Myprojects/Audio-Exercise/test_set_noise.wav")
#rate,noise = wavfile.read("/Users/shahules/Myprojects/Noise-removal-audio/notebooks/noise.wav")
audio = audio/2**15
Audio(audio,rate=rate)

In [60]:
noise = get_noise(len(audio),3000,10000,rate)*10
Audio(noise,rate=rate)

In [62]:
correpted_audio = audio+noise

In [63]:
Audio(data=correpted_audio,rate=rate)

In [64]:
filted = remove_noise(audio_signal=correpted_audio,noise_signal=noise)

In [55]:
Audio(data=filted,rate=rate)